# Gradio.app
[Quickstart](https://www.gradio.app/guides/quickstart)

## Basic input/output example 
e.g. for demoing API calls that normally don't have a UI

In [ ]:
import gradio as gr

def greet(name, date):
    if date:
        return "Hello " + name + "! Happy " + date + "!"

    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs=["text", "text"], outputs="text")
    
demo.launch()   

## Chat interface example

In [ ]:
import gradio as gr

def response(message, history):

    # whatever you want to do with the message and history
    
    return "Someone needs to implement response()!"

demo = gr.ChatInterface(response)

demo.launch()

## Basic Q&A with OpenAI API

In [ ]:
import gradio as gr
import openai

# PUT YOUR CONFIG HERE
AZURE_OPENAI_SERVICE = "my-azure-openai-service-name"
AZURE_OPENAI_DEPLOYMENT = "my-openai-deployment-name"
AZURE_OPENAI_MODEL = "gpt-35-turbo"
AZURE_OPENAI_KEY = "my-azure-openai-key"

# Configure OpenAI
openai.api_type = "azure"
openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
openai.api_version = "2023-05-15"
openai.api_key = AZURE_OPENAI_KEY

def complete(system_prompt, question, temperature):
    if question == "":
        return "Please ask a question.", 0

    messages = []
    messages.append({"role" : "system", "content" : system_prompt})
    messages.append({"role" : "user", "content" : question})

    response = openai.ChatCompletion.create(
                    deployment_id=AZURE_OPENAI_DEPLOYMENT,
                    model=AZURE_OPENAI_MODEL,
                    messages=messages, 
                    temperature=0.7, 
                    max_tokens=1024, 
                    n=1)

    answer = response.choices[0].message.content
    tokens_used = response.usage.completion_tokens

    return answer, tokens_used

# create an example for the user
examples = [["You are a pirate. You make pirate jokes.", "Please write me a professional business plan.", 1]]

demo = gr.Interface(fn=complete, 
                    inputs=[gr.Textbox(label="System Prompt"), 
                            gr.Textbox(label="Question"), 
                            gr.Slider(0, 1, 0.7)], 
                    outputs=[gr.TextArea(label="Response"), 
                             gr.Textbox(label="Tokens used")],
                    examples=examples,
                    title="Basic OpenAI Q&A")
    
demo.launch()   

## Basic chat with OpenAI

In [ ]:
import gradio as gr
import openai

# PUT YOUR CONFIG HERE
AZURE_OPENAI_SERVICE = "my-azure-openai-service-name"
AZURE_OPENAI_DEPLOYMENT = "my-openai-deployment-name"
AZURE_OPENAI_MODEL = "gpt-35-turbo"
AZURE_OPENAI_KEY = "my-azure-openai-key"

# Configure OpenAI
openai.api_type = "azure"
openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
openai.api_version = "2023-05-15"
openai.api_key = AZURE_OPENAI_KEY

system_prompt = """
You are a pirate. You make pirate jokes.
"""

def response(message, history):

    if message == "":
        return

    # build the message history in the ChatCompletion format
    gpt_messages = []
    gpt_messages.append({"role" : "system", "content" : system_prompt})
    for human, assistant in history:
        gpt_messages.append({"role": "user", "content": human })
        gpt_messages.append({"role": "assistant", "content":assistant})
    
    # now add the latest message
    gpt_messages.append({"role": "user", "content": message})

    response = openai.ChatCompletion.create(
                    deployment_id=AZURE_OPENAI_DEPLOYMENT,
                    model=AZURE_OPENAI_MODEL,
                    messages=gpt_messages, 
                    temperature=0.7,
                    stream=True
                    )

    partial_message = ""
    for chunk in response:
        if 'content' in chunk.choices[0].delta:
            partial_message = partial_message + chunk['choices'][0]['delta']['content']
            yield partial_message

gr.ChatInterface(response).queue().launch()
